## 시간 가중 벡터저장소 리트리버
## (TimeWeightedVectorStoreRetriever)

In [1]:
!pip install -qU deeplake lark

In [2]:
from datetime import datetime, timedelta

import faiss
from langchain.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

### 낮은 감쇠율(low decay_rate)

- decay rate 가 낮다는 것은 (여기서는 극단적으로 0에 가깝게 설정할 것입니다) 기억이 더 오래 "기억될" 것임을 의미합니다.

- decay rate 가 0 이라는 것은 기억이 절대 잊혀지지 않는다는 것을 의미하며, 이는 이 retriever를 vector lookup과 동등하게 만듭니다.

In [4]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [5]:
# 임베딩 모델을 정의합니다.
embeddings_model = OpenAIEmbeddings()
# 벡터 저장소를 빈 상태로 초기화합니다.
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})
# 시간 가중치가 적용된 벡터 저장소 검색기를 초기화합니다.
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore, decay_rate=0.0000000000000000000000001, k=1
)

In [6]:
yesterday = datetime.now() - timedelta(days=1)  # 어제 날짜를 계산합니다.
retriever.add_documents(
    # "hello world" 내용의 문서를 추가하고, 메타데이터에 어제 날짜를 설정합니다.
    [Document(page_content="hello world", metadata={
              "last_accessed_at": yesterday})]
)
# "hello foo" 내용의 문서를 추가합니다.
retriever.add_documents([Document(page_content="hello foo")])

['d5747d88-87aa-469c-837c-9ba0fee0f1ca']

retriever.get_relevant_documents("hello world") 메서드를 호출하여 "hello world"와 관련된 문서를 검색합니다.

- "Hello World"가 가장 먼저 반환되는데, 이는 가장 두드러진(salient) 문서이기 때문입니다.
- decay_rate 가 0에 가깝기 때문 에 "Hello World" 문서는 여전히 최신(recent)으로 간주됩니다.

In [7]:
# "Hello World"가 가장 먼저 반환되는 이유는 가장 두드러지기 때문이며, 감쇠율이 0에 가깝기 때문에 여전히 최신 상태를 유지하고 있음을 의미합니다.
retriever.get_relevant_documents("hello world")

c:\Users\anHye\anaconda3\envs\rag\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'last_accessed_at': datetime.datetime(2024, 8, 4, 11, 52, 46, 493609), 'created_at': datetime.datetime(2024, 8, 4, 11, 52, 11, 540298), 'buffer_idx': 0}, page_content='hello world')]

### 높음 감쇠율(high decay_rate)

In [8]:
# 임베딩 모델을 정의합니다.
embeddings_model = OpenAIEmbeddings()
# 벡터 저장소를 빈 상태로 초기화합니다.
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})
# 시간 가중치가 적용된 벡터 저장소 검색기를 초기화합니다.
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore, decay_rate=0.999, k=1
)

In [9]:
yesterday = datetime.now() - timedelta(days=1)  # 어제 날짜를 계산합니다.
retriever.add_documents(
    # "hello world" 내용의 문서를 추가하고, 메타데이터에 어제 날짜를 설정합니다.
    [Document(page_content="hello world", metadata={
              "last_accessed_at": yesterday})]
)
# "hello foo" 내용의 문서를 추가합니다.
retriever.add_documents([Document(page_content="hello foo")])

['9003d9b1-d2e5-4d30-b603-a0c2c0f74d03']

- retriever.get_relevant_documents("hello world") 를 호출하면 "Hello Foo"가 먼저 반환됩니다.
- 이는 retriever가 "hello world"와 관련된 문서를 대부분 잊어버렸기 때문입니다.

In [10]:
# "hello world"와 가장 관련된 문서를 검색합니다.
# "Hello Foo"가 먼저 반환되는 이유는 "hello world"가 대부분 잊혀졌기 때문입니다.
retriever.get_relevant_documents("hello world")

[Document(metadata={'last_accessed_at': datetime.datetime(2024, 8, 4, 11, 54, 23, 454142), 'created_at': datetime.datetime(2024, 8, 4, 11, 53, 47, 913325), 'buffer_idx': 1}, page_content='hello foo')]

### 감쇠율(decay_rate) 정리

- decay_rate 를 0.000001 로 매우 작게 설정한 경우: 감쇠율(즉, 정보를 망각하는 비율)이 매우 낮기 때문에 정보를 거의 잊지 않습니다. 따라서, 최신 정보이든 오래된 정보든 시간 가중치 차이가 거의 없습니다. 이럴때는 유사도에 더 높은 점수를 주게 됩니다.
- decay_rate 를 0.999 로 1에 가깝게 설정한 경우: 감쇠율(즉, 정보를 망각하는 비율)이 매우 높습니다. 따라서, 과거의 정보는 거의다 잊어버립니다. 따라서, 이러한 경우는 최신 정보에 더 높은 점수를 주게 됩니다.

### 가상의 시간(Virtual time)

LangChain의 일부 유틸리티를 사용하면 시간 구성 요소를 모의(mock) 테스트 할 수 있습니다.

mock_now 함수는 LangChain에서 제공하는 유틸리티 함수로, 현재 시간을 모의(mock)하는 데 사용됩니다.

In [11]:
import datetime

from langchain.utils import mock_now

# 현재 시간을 특정 시점으로 설정
mock_now(datetime.datetime(2023, 6, 8, 0, 0, 0))

# 현재 시간 출력
print(datetime.datetime.now())

2024-08-04 11:55:44.119608


mock_now 함수를 사용하여 현재 시간을 2024년 3월 28일 10시 11분으로 설정합니다.

- retriever.get_relevant_documents 메서드를 호출하여 "hello world"라는 쿼리에 대한 관련 문서를 검색합니다.
- 검색 결과를 출력합니다. 이때 문서의 마지막 접근 시간이 설정된 시간(2024년 3월 28일 10시 11분)으로 표시됩니다.

In [14]:
# 현재 시간을 2024년 8월 4일 10시 11분으로 설정합니다.
with mock_now(datetime.datetime(2024, 8, 4, 10, 11)):
    # "hello world"와 관련된 문서를 검색하고 출력합니다.
    print(retriever.get_relevant_documents("hello world"))

[Document(metadata={'last_accessed_at': MockDateTime(2024, 8, 4, 10, 11), 'created_at': datetime.datetime(2024, 8, 4, 11, 53, 47, 913325), 'buffer_idx': 1}, page_content='hello foo')]
